# Getting the info that the client needs for D3.js

Often it is easier to transform data in the backend to be used by D3. In general all data in D3 needs to be a list of dicts. In this example I'm using an existing `world` object to demonstrate the methods that do these transformations. 

In [1]:
#I'm mapping to the actual files in the repo so that I can also use this to troubleshoot
import os
import sys
import numpy as np
import pandas as pd
import pickle
import django
sys.path.append('../..')
#Loading my project settings from prodweb. This allows me to load and query models. 
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'prodweb.settings')
django.setup()


with (open("../pickles/billmanhworld.pkl", 'rb')) as pickle_file:
    world = pickle.load(pickle_file)

In [2]:
sys.path.append('..')
from lib.builders import world as w

`w.get_area_data(world)` gets the area relevant to where the character is. This includes surrounding areas.

In [3]:
world

In [4]:
print(world.Character.location)
pd.DataFrame(w.get_area_data(world)).T

[38, 35]


,aware,danger,elevation,feature,key,nation,nation number,rainfall,terrain,turn_last_visited,visited,x,y
area,1,0.782,17,Earthmore,38:35,Fallforth,1,1,town,10,1,38,35
NArea,1,0.046,12,none,38:34,Fallforth,1,1,desert,5,1,38,34
SArea,1,0.632,17,none,38:36,Fallforth,1,1,desert,0,0,38,36
EArea,1,0.954,11,none,39:35,Fallforth,1,0,desert,0,0,39,35
WArea,1,0.853,25,none,37:35,Fallforth,1,2,mountain,0,0,37,35


For other game principals, it's easy to just make regular `pandas` and `numpy` queries on those datasets. 

In [5]:
world.df_features.loc[world.Character.get_location_key(),'visited'] >= 1

True

Querying the terrain is easy as well. For example, here is a query of all of the world where the _terrain_ is _desert_ and the _danger_ is above .5. These queries are fast and cheap. 

In [6]:
world.df_features.loc[(world.df_features['terrain']=='desert') & 
                     (world.df_features['danger']>=.5)].head()

,y,rainfall,x,key,elevation,terrain,feature,nation number,nation,visited,aware,turn_last_visited,danger
key,,,,,,,,,,,,,
0:15,15,0.0,0,0:15,17.0,desert,NaN,3,Heartpike,0,0,0,0.910
0:16,16,1.0,0,0:16,9.0,desert,NaN,3,Heartpike,0,0,0,0.823
38:32,32,1.0,38,38:32,8.0,desert,NaN,1,Fallforth,0,1,0,1.083
38:33,33,1.0,38,38:33,11.0,desert,NaN,1,Fallforth,1,1,4,0.806
39:33,33,0.0,39,39:33,9.0,desert,NaN,1,Fallforth,1,1,3,0.975


You can also _mask_ the terrain where the character has no knowledge. 

In [7]:
masked_world = w.mask_unknown(world)
world.Character.title = "Noble"
masked_world.head()

,y,rainfall,x,key,elevation,terrain,feature,nation number,nation,visited,aware,turn_last_visited,danger
key,,,,,,,,,,,,,
0:0,0,5.0,0,0:0,2.0,ocean,NaN,3,Heartpike,0,1,0,0.286
1:0,0,6.0,1,1:0,1.0,ocean,NaN,3,Heartpike,0,1,0,0.018
2:0,0,6.0,2,2:0,2.0,ocean,NaN,3,Heartpike,0,1,0,-0.409
3:0,0,6.0,3,3:0,0.0,ocean,NaN,3,Heartpike,0,1,0,-0.808
4:0,0,6.0,4,4:0,1.0,ocean,NaN,3,Heartpike,0,1,0,0.777


## Analysis via list comprehension

Querying people is also easy, but you need to use _list comprehension_ to do it. This is the general structure. 

In [8]:
world.people


[Lecinnech the Speaker of Clearcraft,
 Bohorkach the Speaker of Lilyrun,
 Stufillelle the Speaker of Starforth,
 Erongek the Speaker of Earthcatch,
 Xahillad the Speaker of Maljaw,
 Scamsam the commoner,
 Elennik the Speaker of Leafforth,
 Romendot the Speaker of Malblade,
 Anirandur the commoner,
 Scopeltob the Speaker of Soilring,
 Bemmek the Speaker of Warjaw,
 Stiwillath the commoner,
 Drospus the commoner,
 Scamattat the Speaker of Warfellow,
 Illiamendess the Speaker of Foundersvale,
 Jarpen the commoner,
 Zhoppadle the commoner,
 Stiyellod the Speaker of Spincrest,
 Fannel the Speaker of Snakemelt,
 Elopper the Speaker of Servantslair,
 Erflelton the commoner,
 Anikracka the commoner,
 Fashish the Speaker of Earthmore,
 Drogordosh the Speaker of Heartlight,
 Shecarrob the Speaker of Fallspring,
 Drapeltunt the commoner,
 Xippad the commoner,
 Betallep the commoner,
 Akikrackundle the commoner,
 Shakkish the commoner,
 Praggut the Speaker of Snowmore,
 Drodren the Speaker of Badg

In [15]:
world.year

1134

In [14]:
[t.founded for t in world.towns]

[1000,
 1001,
 1002,
 1003,
 1004,
 1005,
 1006,
 1007,
 1008,
 1009,
 1010,
 1011,
 1012,
 1013,
 1014,
 1015,
 1016,
 1017,
 1018,
 1019,
 1020,
 1021,
 1022,
 1023]

Query the people in the town by setting conditions in the list. These people are the highest in temperment. 

In [ ]:
[[(person.name,person.temperment) for person in t.population if person.temperment>.5] for t in world.towns]

You can also get summary stats on towns. For example, the average temperment per town.

In [ ]:
[(t.name,np.mean([person.temperment for person in t.population])) for t in world.towns]

For some purposes you may want to get a collection of people and edit them in bulk. This is also possible using list comprehension. 

In [ ]:
w.get_people_where_char_has_visited(world)

In [ ]:
r = w.get_people_where_char_has_visited(world)[0]
r

In [ ]:
r['town'].nation

In [ ]:
w.get_relationships_node_map(world)

## Getting map data (mapData): 

In [ ]:
def coordkey(coord):
    '''
    takes a coord [1,1], returns key "1:1"
    '''
    key = ":".join([str(i) for i in coord])
    return key


def get_features_or_NA(world,coord):
    try:
        l = world.df_features.loc[
                chordKey([
                    int(coord[0]),int(coord[1])
                    ])
                ].fillna("none").to_dict(),
    except: 
        l = [{"terrain":"void"}]
    return l[0]


def get_area_data(world):
    """
    note: requires Character
    """
    mapData = {}
    coord = world.Character.location
    key = world.Character.get_location_key()
    l = world.df_features.loc[key]
    mapData = {'area':world.df_features.loc[key].fillna("none").to_dict(),
              'NArea':get_features_or_NA(world,[coord[0],coord[1]-1]),
              'SArea':get_features_or_NA(world,[coord[0],coord[1]+1]),
              'EArea':get_features_or_NA(world,[coord[0]+1,coord[1]]),
              'WArea':get_features_or_NA(world,[coord[0]-1,coord[1]])}
    return mapData


get_area_data(world)

In [ ]:
world.df_features.loc[
                chordKey([
                    int(coord[0]),int(coord[1])
                    ])
                ]

In [ ]:
get_features_or_NA(world,[coord[0],coord[1]-1])

In [ ]:
nations_where_character_has_been = np.unique(where_the_char_has_been['nation'].dropna())
nations_where_character_has_been

You can use this to get lists of people who the character knows. 

In [ ]:
[[t for t in n.get_all_towns(world)] for n in world.nations if n.name in nations_where_character_has_been]

In [ ]:
[[t for t in T.population] for T in world.towns if T.nation in nations_where_character_has_been]

In [ ]:
[(T.nation,T.name) for T in world.towns]

In [ ]:
[[t.get_person_data() for t in T.population] for T in world.towns if T.nation in nations_where_character_has_been]

In [ ]:
masked_world.loc[world.Character.get_location_key()]

You can query nations or towns. `world.nations` is a list of _nations_ and `world.towns` is a list of _towns_

In [ ]:
where_the_char_has_been = world.df_features.loc[(world.df_features['visited']==1)&
                                                (world.df_features['terrain']=='town')]
where_the_char_has_been

## Getting the diplomacy and relationships data for the journal page

In [ ]:
rel = w.get_people_where_char_has_visited(world)
rel

In [ ]:
w.get_relationships_node_map(world)

In [ ]:
hierarchy = [{'nation':i.name,
              'children':i.diplomacy} for i in world.nations]



In [ ]:
p = rel[0]['town'].population[0]


In [ ]:
def get_people_where_char_has_visited(world):
    """
    returns a dictionary of {{"town":[<obj>],"people":[<obj>]}
    filtered to where the character has been. 
    """
    where_the_char_has_been = world.df_features.loc[(world.df_features['visited']==1)&
                                                    (world.df_features['terrain']=='town')].dropna()
    towns_and_people = [{"town":T,
                         "people":[t for t in T.population]} 
                        for T in world.towns if T.name in np.unique(w.where_the_char_has_been['feature'])]
    return towns_and_people

def get_relationships_node_map(world):
    nodes = [{'name':r['town'].name,
              'title':str(r['town']).split(":")[0],
              'nation':r['town'].diplomacy['nation'],
              'population':r['town'].pop,
              'type':r['town'].type,
              'location':r['town'].key,
              'founded year':r['town'].founded,
              'national fealty':r['town'].diplomacy['national fealty'],
             'children': [{'name':p.name,
                          'role':p.role,
                          'temperment':p.temperment,
                          'loyalty':p.loyalty,
                          'type':'person'} for p in r['people']]} 
             for r in w.get_people_where_char_has_visited(world)]
    return nodes
        
w.get_people_where_char_has_visited(world)

In [ ]:
relationships = w.get_relationships_node_map(world)
relationships

In [ ]:
nations = [s['nation'] for s in relationships]
nations

In [ ]:
[i for i in relationships]

In [ ]:
[{'nation':i,'towns':[t for t in relationships if t['nation']==i]} for i in nations]

In [ ]:
[i for i in nations]